In [5]:
## Data Ingestion
from langchain_community.document_loaders import TextLoader

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [7]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://admissions.ncsu.edu/connect/frequently-asked-questions/#application","https://admissions.ncsu.edu/apply/international/#FAQ",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(                        
                         class_=("accordion-item__label h5","accordion-item__content")
                     )))

text_documents=loader.load()


In [8]:
text_documents

[Document(page_content='Why am I being asked to use a personal email address when completing my application to NC\xa0State?\nWe strongly recommend that applicants use a personal email address when completing their application to NC\xa0State, not a school-issued email address. This helps to ensure that you continue to receive important information about your application status and deadlines even after you graduate from high school or community college.\nHow do I qualify for a fee waiver as a first-year applicant?\nNC\xa0State does not distribute fee waiver codes to applicants; however, there are a variety of ways to qualify for a fee waiver. International applicants are not eligible for fee waivers.\nFee Waiver Qualifications\nTo be eligible for a fee waiver, you must meet one of the qualifications below. \n\nI am eligible to receive a federal Pell grant.\nMy annual family income falls within the Income Eligibility Guidelines set by the USDA Food and Nutrition Service.\nMy family receiv

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(text_documents)
documents[:5]

[Document(page_content='Why am I being asked to use a personal email address when completing my application to NC\xa0State?\nWe strongly recommend that applicants use a personal email address when completing their application to NC\xa0State, not a school-issued email address. This helps to ensure that you continue to receive important information about your application status and deadlines even after you graduate from high school or community college.\nHow do I qualify for a fee waiver as a first-year applicant?\nNC\xa0State does not distribute fee waiver codes to applicants; however, there are a variety of ways to qualify for a fee waiver. International applicants are not eligible for fee waivers.\nFee Waiver Qualifications\nTo be eligible for a fee waiver, you must meet one of the qualifications below.', metadata={'source': 'https://admissions.ncsu.edu/connect/frequently-asked-questions/#application'}),
 Document(page_content='I am eligible to receive a federal Pell grant.\nMy annual

In [10]:
documents

[Document(page_content='Why am I being asked to use a personal email address when completing my application to NC\xa0State?\nWe strongly recommend that applicants use a personal email address when completing their application to NC\xa0State, not a school-issued email address. This helps to ensure that you continue to receive important information about your application status and deadlines even after you graduate from high school or community college.\nHow do I qualify for a fee waiver as a first-year applicant?\nNC\xa0State does not distribute fee waiver codes to applicants; however, there are a variety of ways to qualify for a fee waiver. International applicants are not eligible for fee waivers.\nFee Waiver Qualifications\nTo be eligible for a fee waiver, you must meet one of the qualifications below.', metadata={'source': 'https://admissions.ncsu.edu/connect/frequently-asked-questions/#application'}),
 Document(page_content='I am eligible to receive a federal Pell grant.\nMy annual

In [11]:
## Vector Embedding And Vector Store
from langchain_community.embeddings import OllamaEmbeddings
#from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
embeddings = OllamaEmbeddings(model="nomic-embed-text")
db = Chroma.from_documents(documents,embeddings)

In [12]:
query = "When will I get a decision?"
retireved_results=db.similarity_search(query)
retireved_results
#print(retireved_results[0].page_content)

[Document(page_content='October SAT – Scores should be received in time to be considered for the early action deadline.\nNovember SAT – Scores will not be received in time to be considered for the early action deadline.\nOctober ACT Dates – We cannot guarantee scores from all October test dates will be received in time for consideration if you apply by the early action deadline as it can take 2-8 weeks for scores to be released. We will consider any scores we have on file at the time your application is reviewed.\n\nRegular Decision Deadline\n\nThe December SAT and ACT are the last test dates we will be able to consider for our Regular Decision deadline.', metadata={'source': 'https://admissions.ncsu.edu/connect/frequently-asked-questions/#application'}),
 Document(page_content='When will I get an admission decision?\nFor applicants outside of the U.S., decisions are made on a rolling basis. In other words, decisions are made as your application becomes complete and is reviewed. Applic

In [13]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [14]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [15]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [16]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000230F90731C0>)

In [17]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [20]:
response=retrieval_chain.invoke({"input":"How should I send my SAT scores?"})

In [22]:
response['answer']

'To send your SAT scores to North Carolina State University (NC State), you can follow these steps:\n\n1. Sign in to My AP using your College Board username and password.\n2. Go to My AP Profile and select "Score Send."\n3. Type in North Carolina State University when it appears on the list, and hit save.\n\nIt\'s important to note that international students are not required to submit SAT or ACT test scores, but if they wish to be considered for scholarship opportunities, they should submit an application via Pack ASSIST, NC State\'s scholarship portal, as soon as possible. Additionally, applicants attending school inside the US with a weighted GPA of 2.5 or higher but less than 2.8 will be required to self-report a standardized test score with their application (SAT or ACT). However, verified test scores will be required for enrollment, and NC State will not accept standardized test scores on your high school transcript.'